In [7]:
import numpy as np
import open3d as o3d
from sklearn.cluster import MeanShift
import trimesh
from utils import trimesh_to_o3d, o3d_to_trimesh

In [ ]:
def local_spacing(pcd, k=25):
    # estimate D(p) for every point
    tree = o3d.geometry.KDTreeFlann(pcd)
    D = np.empty(len(pcd.points))
    for idx, p in enumerate(pcd.points):
        _, idxs, dists = tree.search_knn_vector_2d(p, k)
        D[idx] = np.sqrt(dists[-2])        # r = max distance
    return D

def trilateral_shift(pcd, scan_id, D, alpha=1.0, sigma_factor=1.0):
    pts = np.asarray(pcd.points);  nrm = np.asarray(pcd.normals)
    new_pts = pts.copy()
    for i, (p, n, d) in enumerate(zip(pts, nrm, D)):
        r = alpha * d;  h = 2*alpha*d
        cyl_mask = np.logical_and.reduce([
            np.linalg.norm(np.cross(pts - p, n), axis=1) < r,   # radial
            np.abs((pts - p) @ n) < h/2,                        # height
            scan_id != scan_id[i]                               # other scan
        ])
        idx = np.where(cyl_mask)[0]
        if idx.size == 0: continue
        P = pts[idx]; N = nrm[idx]
        ri = np.linalg.norm(P - p - ((P - p) @ n)[:,None]*n, axis=1)
        hi = (P - p) @ n
        rho = 1./(D[idx]**2); rho_max = rho.max()
        w = np.exp(-ri**2/(2*r**2*sigma_factor**2)) \
            * np.exp(-hi**2/(2*h**2*sigma_factor**2)) \
            * np.exp(-(rho-rho_max)**2/(2*(rho_max-rho.min())**2*sigma_factor**2))
        shift = (w*hi).sum() / (w.sum() + 1e-12)
        new_pts[i] = p + shift * n
    pcd.points = o3d.utility.Vector3dVector(new_pts)
    return pcd

In [ ]:
import numpy as np
import open3d as o3d
from typing import Tuple, Union

def local_sampling_spacing(
        query: np.ndarray,
        points: Union[o3d.geometry.PointCloud, np.ndarray],
        k: int = 25,
        tree: o3d.geometry.KDTreeFlann = None
) -> Tuple[np.ndarray, np.ndarray]:
    """Estimate local sample spacing D(p_j) around each of the query points p_j.

    Args:
        query (np.ndarray): array of query points
        points (np.ndarray): array of all points in the pointcloud
        k (int, optional): number of nearest neighbours to find. Defaults to 25.
        tree (o3d.geometry.KDTreeFlann, optional): tree to seach for nearest neighbours. Defaults to None.

    Raises:
        ValueError: points must be a (N,3) array
        ValueError: query must be a (N,3) array
        ValueError: there must be at least k neighbours in the pointcloud

    Returns:
        Tuple[np.ndarray, np.ndarray]: local sample spacings, local sample densities
    """
    if not (isinstance(points, o3d.geometry.PointCloud) or isinstance(points, np.ndarray)):
        raise TypeError("points must be either an o3d pointcloud or a numpy ndarray")
    if isinstance(points, np.ndarray) and (points.ndim != 2 or points.shape[1] != 3):
        raise ValueError("points must be a (N,3) array")
    if query.ndim != 2 or query.shape[1] != 3:
        raise ValueError("query must be a (N,3) array")

    if tree is None:
        tree = o3d.geometry.KDTreeFlann(points.transpose())

    q = query.reshape(-1, 3)
    n_query = q.shape[0]

    D = np.empty(n_query)
    rho = np.empty(n_query)

    for i, p in enumerate(q):
        # k + 1 because the first hit is p itself with distance=0
        _, idx, d2 = tree.search_knn_vector_3d(p, k + 1)

        if len(idx) < k + 1:
            raise ValueError(f"Found only {len(idx)-1} neighbours; "
                             f"increase point cloud size or reduce k.")

        r_k2 = d2[-1]               # Squared distance to k-th real neighbour
        rho[i] = k / (np.pi * r_k2)
        D[i]  = 1.0 / np.sqrt(rho[i])

    # Return scalars if a single point was given
    if query.ndim == 1 or query.shape[0] == 1:
        return D[0], rho[0]
    return D, rho


In [11]:
mesh1 = trimesh.load_mesh('meshes/bottle_1.ply')
mesh2 = trimesh.load_mesh('meshes/bottle_2.ply')

mesh1_o3d = trimesh_to_o3d(mesh1)
mesh2_o3d = trimesh_to_o3d(mesh2)

pcd_1 = o3d.geometry.PointCloud()
pcd_2 = o3d.geometry.PointCloud()
pcd_1.points = mesh1_o3d.vertices
pcd_2.points = mesh2_o3d.vertices

In [30]:
print(local_spacing(pcd_1))
print(local_sampling_spacing(np.asarray(pcd_1.points), np.asarray(pcd_1.points)))

[0.00809955 0.00878236 0.01072631 ... 0.00753483 0.00774    0.00787873]
(array([0.00287345, 0.00320869, 0.00384399, ..., 0.0026871 , 0.0028199 ,
       0.00286917]), array([121113.64897006,  97128.27252865,  67676.25579015, ...,
       138494.66088189, 125756.90610107, 121474.83015164]))
